In [47]:
from citableclass import Credentials, Citable, Json2DF
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
#title = Credentials(authors='olivier.defaux,gerd.grasshoff,malte.vogl', project='geography',title='Maps of the Iberian Peninsula')
#title.titlepage()

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#from utility import plot_maps,comp_plot_map_01,comp_plot_map_02
from tools import Geography

Loading BokehJS ...

In [50]:
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.io import output_notebook, output_file, save
output_notebook()
from bokeh.models import HoverTool,BoxZoomTool,ResetTool, WheelZoomTool
from bokeh.layouts import row, gridplot, layout

Loading BokehJS ...

In [51]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('max_colwidth', 260)

In [52]:
options = {"compact": True, "bg": "#09a3d5",
           "color": "white", "font": "Source Sans Pro","collapse_phrases":False}

On the basis of both recensions of the Geography, the localities of the Iberian peninsula are plotted and compared.

### Load resources

In [53]:
OmegaJ = pd.read_json('../data/OmegaStructure.json')
XiJ = pd.read_json('../data/XiStructure.json')

In [54]:
%run geographytools.py

In [55]:
def findTS(s):
    out=""
    for i,r in om.iterrows():
        sci=r["sec_ID"]
        if sci in s:
            out=r["type_sec"]
            exit
    return(out)
findTS("2.04.01.01")

'area presentation'

In [56]:
def Js2Geodf(df): 
# df=OmegaJ["chapters"][0]
    om=json_normalize(df,"section")
    om=om.dropna(subset=["sec_part"])
    l=[]
    for i,x in om.iterrows():
        k={"type_sec":x["type_sec"]}
        l.append([x["sec_part"]])
#
    listItems=list(flatten_list(l))
    dfout=pd.DataFrame(listItems)
    dfout["type_sec"]=dfout.apply(lambda x: findTS(x["ID"]),axis=1)
    return(dfout)

In [57]:
df=OmegaJ["chapters"][0]
om=json_normalize(df,"section")
om.head(2)

,sec_ID,type_sec,text,sec_part
0,2.04.00,title,Ἰσπανίας Βαιτικῆς θέσις,NaN
1,2.04.01,area presentation,NaN,"[{'ID': '2.04.01.01', 'type': 'text string', 'text': 'τῆς Ἰσπανίας κατὰ δὲ Ἕλληνας Ἰβηρίας τρεῖς εἰσιν ἐπαρχίαι·'}, {'ID': '2.04.01.02', 'type': 'text string', 'text': 'Βαιτικὴ καὶ Λουσιτανία καὶ Ταρρακωνησία'}]"


In [58]:
dfOmega=Js2Geodf(OmegaJ["chapters"][0])
dfXi=Js2Geodf(XiJ["chapters"][0])

In [59]:
dfOmega.head(2)

,ID,type,text,category,people,toponym,coord,type_sec
0,2.04.01.01,text string,τῆς Ἰσπανίας κατὰ δὲ Ἕλληνας Ἰβηρίας τρεῖς εἰσιν ἐπαρχίαι·,NaN,NaN,NaN,NaN,area presentation
1,2.04.01.02,text string,Βαιτικὴ καὶ Λουσιτανία καὶ Ταρρακωνησία,NaN,NaN,NaN,NaN,area presentation


### Translate greek to decimal coordinates

In [60]:
def reformatCoord(row,longLat, xy='coord_x'):
    if type(row[xy]) == dict:
        return (row[xy][longLat]['integer'].strip(),row[xy][longLat]['fraction'].strip())
    else:
        return False

In [61]:
gfrac={"":0,"Lς":2/3,"ςL":2/3,"ιβ":1/12,"ς":1/6,"δ":1/4,"γ":1/3,"γιβ":5/12,"L":1/2,"Lιβ":7/12,"γο":2/3,"Lδ":3/4,"Lγ":5/6,"Lγιβ":11/12,"η":1/8}
gint={"":0,"α":1,"β":2,"γ":3,"δ":4,"ε":5,"ς":6,"ζ":7,"η":8,"θ":9,"ι":10,"κ":20,"λ":30,"μ":40}

In [62]:
def translate(row):
    if type(row)==tuple:
        try:
            temp_frac = gfrac[row[1]]
        except:
            temp_frac = gint[row[1]]
        try:
            if len(row[0]) == 1:
                temp_int = gint[row[0]]
            elif len(row[0]) == 2:
                temp_int = gint[row[0][0]] + gint[row[0][1]]
            elif row[0] == '':
                temp_int = 0
        except:
            temp_int = None
        return temp_int + temp_frac

Omega recension

In [63]:
dfTemp = dfOmega.copy()
dfTemp['longitude'] = dfOmega.apply(lambda row: reformatCoord(row,'long','coord'),axis=1).apply(translate)
dfTemp['latitude'] = dfOmega.apply(lambda row: reformatCoord(row,'lat','coord'),axis=1).apply(translate)

In [64]:
dfTemp.head(20)

,ID,type,text,category,people,toponym,coord,type_sec,longitude,latitude
0,2.04.01.01,text string,τῆς Ἰσπανίας κατὰ δὲ Ἕλληνας Ἰβηρίας τρεῖς εἰσιν ἐπαρχίαι·,NaN,NaN,NaN,NaN,area presentation,NaN,NaN
1,2.04.01.02,text string,Βαιτικὴ καὶ Λουσιτανία καὶ Ταρρακωνησία,NaN,NaN,NaN,NaN,area presentation,NaN,NaN
2,2.04.02.01,text string,καὶ τῆς μὲν Βαιτικῆς ἡ πρὸς δυσμὰς καὶ ἄρκτους πλευρὰ ἀφορίζεται τῇ τε Λουσιτανίᾳ· καὶ τῆς Ταρρακωνησίας μέρει,NaN,NaN,NaN,NaN,borders description,NaN,NaN
3,2.04.02.02,text string,ἧς πλευρᾶς ἡ περιγραφὴ ἔχει οὕτως·,NaN,NaN,NaN,NaN,borders description,NaN,NaN
4,2.04.03.01,locality,NaN,"[river mouth, boundary]",Turdetani,<τὸ δυσμικώτερον στόμα Ἄνα τοῦ ποταμοῦ>,"{'long': {'integer': 'δ', 'fraction': 'ιβ'}, 'lat': {'integer': 'λζ', 'fraction': 'γο'}}",coast section,4.083333,37.666667
5,2.04.03.02,locality,NaN,river mouth,Turdetani,τὸ ἀνατολικώτερον στόμα Ἄνα τοῦ ποταμοῦ,"{'long': {'integer': 'δ', 'fraction': 'γ'}, 'lat': {'integer': 'λζ', 'fraction': 'L'}}",coast section,4.333333,37.500000
6,2.04.03.03,locality,NaN,river path,,ἡ πρὸς ἀνατολὰς τοῦ ποταμοῦ ἐπιστροφή,"{'long': {'integer': 'ς', 'fraction': 'γ'}, 'lat': {'integer': 'λθ', 'fraction': ''}}",coast section,6.333333,39.000000
7,2.04.03.04,locality,NaN,"[river path, boundary]",,τὸ πρὸς τῷ πέρατι τῆς Λουσιτανίας τοῦ ποταμοῦ μέρος,"{'long': {'integer': 'θ', 'fraction': ''}, 'lat': {'integer': 'λθ', 'fraction': ''}}",coast section,9.000000,39.000000
8,2.04.03.07,locality,NaN,boundary,,καὶ ἡ ἐντεῦθεν γραμμὴ γραφομένη παρὰ τὴν Ταρρακωνησίαν ὡς ἐπὶ τὸ Βαλιαρικὸν πέλαγος πέρας ἔχουσα,"{'long': {'integer': 'ιβ', 'fraction': ''}, 'lat': {'integer': 'λζ', 'fraction': 'δ'}}",coast section,12.000000,37.250000
9,2.04.03.08,locality,NaN,river source,,αἱ δὲ πηγαὶ τοῦ ποταμοῦ ἐπέχουσι μοίρας,"{'long': {'integer': 'ιδ', 'fraction': ''}, 'lat': {'integer': 'μ', 'fraction': ''}}",coast section,14.000000,40.000000


Xi recension

In [65]:
dfTempX = dfXi.copy()
dfTempX['longitude'] = dfXi.apply(lambda row: reformatCoord(row,'long','coord'),axis=1).apply(translate)
dfTempX['latitude'] = dfXi.apply(lambda row: reformatCoord(row,'lat','coord'),axis=1).apply(translate)

# Data preparation for the map drawing

### Coasts and boundaries (Omega)

Selection of coastal localities

In [66]:
f = dfTemp[(dfTemp.type_sec == 'coast section') & (dfTemp.type == 'locality') & dfTemp.category.apply(lambda row: row not in ['boundary','river path','river source'])][['longitude','latitude']]

In [67]:
f;

Coastal lines

In [68]:
#fbae = coast of Baetica, omega
baria = dfTemp[dfTemp.toponym == 'ἐφʹ ἧς Βαρεία πόλις '][['longitude','latitude']]
fbae = f.iloc[0:28]
fbae = fbae.append(baria)
fbae = fbae.values

In [69]:
#flus = coast of Lusitania, omega
flus = f.iloc[28:42].append(f.iloc[0:1]).sort_index(ascending=True)
flus = flus.values

In [70]:
#ftar = coast of Tarraconensis, omega
ftar1 = f.iloc[42:70].append(f.iloc[41:42]).sort_index(ascending=True)
ftar1 = ftar1.values
ftar2 = f.iloc[70:].sort_index(ascending=False)
baebound = dfTemp[dfTemp.toponym == 'μετὰ τὸ πρὸς τῇ Βαιτικῇ πέρας'][['longitude','latitude']]
ftar2 = ftar2.append(baebound).append(baria)
ftar2 = ftar2.values

In [71]:
dfTemp;

### Coasts and boundaries (Xi)

Selection of coastal localities

In [72]:
g = dfTempX[(dfTempX.type_sec == 'coast section') & (dfTempX.type == 'locality') & dfTempX.category.apply(lambda row: row not in ['boundary','river path','river source'])][['longitude','latitude']]

Coastal lines

In [73]:
#gbae = coast of Baetica, xi
bariaX = dfTempX[dfTempX.toponym == 'ἐφʹ ἧς Βαραλία πόλις'][['longitude','latitude']]
gbae = g.iloc[0:26]
gbae = gbae.append(bariaX)
gbae = gbae.values

In [74]:
#glus = coast of Lusitania, xi
glus = g.iloc[27:41].append(g.iloc[0:1]).sort_index(ascending=True)
glus = glus.values

In [75]:
#gtar = coast of Tarraconensis, xi
gtar1 = g.iloc[41:68].append(g.iloc[40:41]).sort_index(ascending=True)
gtar1 = gtar1.values
gtar2 = g.iloc[69:].sort_index(ascending=False)
baeboundX = dfTempX[dfTempX.toponym == 'μετὰ τὸ πρὸς τῇ Βαιτικῇ πέρας'][['longitude','latitude']]
gtar2 = gtar2.append(baeboundX).append(bariaX)
gtar2 = gtar2.values

In [76]:
# make a dictionary for all coasts's lines using Xi for later use in function plot_maps
gs_dict={'gbae' : gbae, 'glus' : glus, 'gtar1' : gtar1, 'gtar2' : gtar2}

In [77]:
# make a dictionary for all coasts's lines using Omega for later use in function plot_maps
fs_dict={'fbae' : fbae, 'flus' : flus, 'ftar1' : ftar1, 'ftar2' : ftar2}

In [78]:
# make a dictionary for all data
df_dict={"dfTemp" : dfTemp,  "dfTempX" : dfTempX}

# Maps

### Omega recension

In [2]:
fig=Geography(fs_dict,gs_dict,df_dict)

NameError: name 'fs_dict' is not defined

In [ ]:
fig.plot_recension(fs_dict,gs_dict,df_dict,select_recension='Omega')


### Xi Recension

In [ ]:
fig.plot_recension(fs_dict,gs_dict,df_dict,select_recension='Xi')

## Comparison Ξ and Ω

In [ ]:
fig.plot_compare_recension(fs_dict,gs_dict,df_dict)
   

In [ ]:
fig.plot_border_compare_recension(fs_dict,gs_dict,df_dict)

# Interactive Overview of maps

In [ ]:
import os 
import  ipywidgets as widgets
from ipywidgets import interact
from IPython.display import Image
import os
# Create widgets
directory = widgets.Dropdown(options=['image'])
images = widgets.Dropdown(options=os.listdir(directory.value))

# Updates the image options based on directory value
def update_images(*args):
    images.options = os.listdir(directory.value)

# Tie the image options to directory value
directory.observe(update_images, 'value')

# Show the images
def show_images(fdir, file):
    display(Image(f'{fdir}/{file}'))

_ = interact(show_images, fdir=directory, file=images)